In [ ]:
import mysql.connector
import json
import random
from datetime import datetime, timedelta

db = mysql.connector.connect(
    host="127.0.0.1",
    user="root",
    password="0000",
    database="test"
)

cursor = db.cursor()

# 1. JSON 파일에서 회원 ID 추출 및 회원 정보 생성

with open('C:/Users/heon9/sample_user_log.json', 'r') as file:
    activity_logs = json.load(file)

# user_id 추출
user_ids = set(log['user_id'] for log in activity_logs)

# 회원 정보 생성 및 삽입
for user_id in user_ids:
    username = user_id
    password_hash = "hashed_password"
    email = f"{user_id}@example.com"
    phone_number = f"010-{random.randint(1000, 9999)}-{random.randint(1000, 9999)}"
    encrypted_data = "encrypted_info"

    cursor.execute("""
        INSERT INTO customers (username, password_hash, email, phone_number, encrypted_data)
        VALUES (%s, %s, %s, %s, %s)
    """, (username, password_hash, email, phone_number, encrypted_data))

db.commit()

print(f"Total {len(user_ids)} users inserted into the customers table.")

# 2. 상품 정보 100개 랜덤 생성

brands = [chr(i) for i in range(65, 91)]  # 브랜드 리스트 만들기
brand_shipping_fees = {brand: round(random.uniform(3.00, 15.00), 2) for brand in brands}

# 브랜드 정보 삽입
for brand in brands:
    cursor.execute("""
        INSERT INTO brands (brand_name, shipping_fee)
        VALUES (%s, %s)
    """, (brand, brand_shipping_fees[brand]))

# 상품 정보 100개 만들기
for _ in range(100):
    product_code = f"P{random.randint(1000, 9999)}_{random.choice(brands)}"
    product_name = f"Product_{random.randint(1, 1000)}"
    size = random.choice(['S', 'M', 'L', 'XL'])
    color = random.choice(['Red', 'Blue', 'Green', 'Black', 'White'])
    stock_quantity = random.randint(1, 100)
    price = round(random.uniform(10.00, 100.00), 2)
    brand_id_query = "SELECT brand_id FROM brands WHERE brand_name = %s"

    cursor.execute(brand_id_query, (product_code.split('_')[1],))
    brand_id = cursor.fetchone()[0]

    cursor.execute("""
        INSERT INTO products (product_code, product_name, size, color, stock_quantity, price, brand_id)
        VALUES (%s, %s, %s, %s, %s, %s, %s)
    """, (product_code, product_name, size, color, stock_quantity, price, brand_id))

db.commit()

print("100 products inserted into the products table.")

# 3. 주문 정보 1000개 랜덤 만들기

start_date = datetime(2024, 1, 1)
end_date = datetime.now()

def random_date(start, end):
    return start + timedelta(seconds=random.randint(0, int((end - start).total_seconds())))

dates = [random_date(start_date, end_date) for _ in range(1000)]
dates.sort()

# 모든 날짜가 최소 하나의 주문을 갖도록 함
order_dates = {}
for date in dates:
    order_date_str = date.strftime('%Y-%m-%d')
    if order_date_str not in order_dates:
        order_dates[order_date_str] = []
    order_dates[order_date_str].append(date)

# 주문 정보 생성
for date_str, order_list in order_dates.items():
    for order_date in order_list:
        customer_id_query = "SELECT customer_id FROM customers ORDER BY RAND() LIMIT 1"
        cursor.execute(customer_id_query)
        customer_id = cursor.fetchone()[0]

        shipping_address = f"{random.randint(1, 100)} Main St, City {random.randint(1, 100)}, Country"
        total_amount = round(random.uniform(20.00, 500.00), 2)
        discount_amount = round(random.uniform(0, 50.00), 2)
        shipping_fee = round(random.uniform(3.00, 15.00), 2)
        order_status = random.choice(['결제전', '결제후', '배송중', '배송완료', '취소'])

        cursor.execute("""
            INSERT INTO orders (customer_id, order_date, shipping_address, total_amount, discount_amount, shipping_fee, order_status)
            VALUES (%s, %s, %s, %s, %s, %s, %s)
        """, (customer_id, order_date, shipping_address, total_amount, discount_amount, shipping_fee, order_status))

        order_id = cursor.lastrowid

        # 랜덤 상품 추가
        num_items = random.randint(1, 5)
        for _ in range(num_items):
            product_id_query = "SELECT product_id FROM products ORDER BY RAND() LIMIT 1"
            cursor.execute(product_id_query)
            product_id = cursor.fetchone()[0]

            quantity = random.randint(1, 5)
            cursor.execute("SELECT price FROM products WHERE product_id = %s", (product_id,))
            unit_price = cursor.fetchone()[0]
            total_price = quantity * unit_price

            cursor.execute("""
                INSERT INTO order_items (order_id, product_id, quantity, unit_price, total_price)
                VALUES (%s, %s, %s, %s, %s)
            """, (order_id, product_id, quantity, unit_price, total_price))

db.commit()

print("1000 orders and related order items inserted into the orders and order_items tables.")

cursor.close()
db.close()
